In [ ]:
from moviepy.editor import VideoFileClip
import numpy as np
import cv2
import librosa
import librosa.display
import matplotlib.pyplot as plt
from google.colab import files
import tempfile
import os

# 動画アップロード
uploaded = files.upload()
video_path = next(iter(uploaded))
print(f"Uploaded: {video_path}")

# 動画→音声を一時ファイルに抽出
clip = VideoFileClip(video_path)
audio = clip.audio
fps = clip.fps
duration = clip.duration
total_frames = int(duration * fps)

temp_wav_path = tempfile.mktemp(suffix=".wav")
audio.write_audiofile(temp_wav_path, fps=44100, verbose=False, logger=None)

# librosaで音声読み込み
y, sr = librosa.load(temp_wav_path, sr=44100)

# パラメータ設定
hop_length = 512
n_fft = 2048
stft = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
spectrogram = np.abs(stft)

# 周波数と時間の配列取得
frequencies = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
times = librosa.frames_to_time(np.arange(spectrogram.shape[1]), sr=sr, hop_length=hop_length)

# 351Hzに最も近いインデックスを探す
target_freq = 351
freq_index = np.argmin(np.abs(frequencies - target_freq))

# 該当周波数の強さを時間ごとに抽出
target_magnitudes = spectrogram[freq_index, :]
max_index = np.argmax(target_magnitudes)
max_time = times[max_index]
max_frame = int(max_time * fps)

print(f"📍 Max 351Hz magnitude at time: {max_time:.2f}s (frame {max_frame})")

# OpenCVで該当フレーム抽出
cap = cv2.VideoCapture(video_path)
cap.set(cv2.CAP_PROP_POS_FRAMES, max_frame)
ret, frame = cap.read()

if ret:
    output_path = "frame_max_351Hz.jpg"
    cv2.imwrite(output_path, frame)
    print(f"✅ Frame with max 351Hz saved as: {output_path}")
    files.download(output_path)
else:
    print("❌ Failed to retrieve the frame.")

cap.release()

# スペクトログラムのdB変換（全体）
D_full = librosa.amplitude_to_db(np.abs(stft), ref=np.max)

plt.figure(figsize=(14, 6))
librosa.display.specshow(D_full, sr=sr, hop_length=hop_length, x_axis='time', y_axis='log', cmap='magma')
plt.colorbar(format='%+2.0f dB')
plt.title("Full Spectrogram with 351Hz Peak Marker")
plt.ylim(300, 400)  # 351Hz付近だけを表示（必要に応じて広げてもOK）
plt.axvline(max_time, color='cyan', linestyle='--', label='max 351Hz')  # ピーク位置に線を引く
plt.legend()
plt.show()